In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import PIL
import cv2
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import warnings

from tensorflow import keras
from PIL import Image
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from keras.optimizers import adam_v2
from keras.metrics import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, BatchNormalization, concatenate, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from keras.callbacks import EarlyStopping
from keras.layers.core import SpatialDropout2D
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from keras import backend as K
from sklearn.model_selection import train_test_split
from warnings import filterwarnings


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#dataset path
train_path = '/content/drive/MyDrive/classes4-data/Train'
test_path = '/content/drive/MyDrive/classes4-data/Test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224),batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224),batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224),batch_size=10, shuffle=True)

Found 18692 images belonging to 4 classes.
Found 18692 images belonging to 4 classes.
Found 4670 images belonging to 4 classes.


In [ ]:
SEED = 123
batch_size = 10
img_height = 224
img_width =  224

In [ ]:
classes=['Benign keratosis BKL',
 'Melanoma MEL',
 'Basal cell carcinoma BCC',
 'Melanocytic Nevi NV']

In [ ]:
import pathlib
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units= 4, activation='sigmoid')
])

In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 200704)            0         
                                                                 
 dense (Dense)               (None, 4)                 8

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    
  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['accuracy'])
    
# Early stopping to stop the training if loss start to increase. It also avoids overfitting.
es = EarlyStopping(patience=3,monitor="val_loss")

In [ ]:
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

#train model
history = model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
                    epochs=10,
                    verbose=2,
                    callbacks=[early_stopping_monitor]
                   )

Epoch 1/10
1870/1870 - 4387s - loss: 0.8321 - accuracy: 0.5825 - val_loss: 0.4212 - val_accuracy: 0.6182 - 4387s/epoch - 2s/step
Epoch 2/10
1870/1870 - 788s - loss: 0.4040 - accuracy: 0.6256 - val_loss: 0.3609 - val_accuracy: 0.6748 - 788s/epoch - 422ms/step
Epoch 3/10
1870/1870 - 785s - loss: 0.3781 - accuracy: 0.6612 - val_loss: 0.3222 - val_accuracy: 0.7323 - 785s/epoch - 420ms/step
Epoch 4/10
1870/1870 - 790s - loss: 0.3235 - accuracy: 0.7263 - val_loss: 0.2519 - val_accuracy: 0.8027 - 790s/epoch - 423ms/step
Epoch 5/10
1870/1870 - 781s - loss: 0.2667 - accuracy: 0.7888 - val_loss: 0.2210 - val_accuracy: 0.8468 - 781s/epoch - 418ms/step
Epoch 6/10
1870/1870 - 780s - loss: 0.2160 - accuracy: 0.8415 - val_loss: 0.1763 - val_accuracy: 0.8809 - 780s/epoch - 417ms/step
Epoch 7/10
1870/1870 - 778s - loss: 0.1940 - accuracy: 0.8711 - val_loss: 0.1399 - val_accuracy: 0.9084 - 778s/epoch - 416ms/step
Epoch 8/10
1870/1870 - 787s - loss: 0.1466 - accuracy: 0.9067 - val_loss: 0.1059 - val_accu

In [ ]:
_, test_accuracy = model.evaluate_generator(test_batches, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

NameError: ignored

In [ ]:
print('test accuracy: %.2f' % (test_accuracy*100))


NameError: ignored